# 这是我的第二个练习

In [367]:
# -*- coding: utf-8 -*-

import requests 
from bs4 import BeautifulSoup
import os
from collections import defaultdict

## 爬取深圳地铁数据

In [368]:
url = "http://jt.sz.bendibao.com/z/metro/linemap.shtml"

In [369]:
# get a Beautifulsoup object info from an url
def get_html(url,encoding='gb2312'):
    xml = requests.get(url)
    xml.encoding = encoding
    soup = BeautifulSoup(xml.text, 'lxml')
    return soup
soup = get_html(url)

In [370]:
all_line = soup.find_all("div",class_="line-list")

l1 = all_line[0]
station_info = l1.find_all('div',class_="station")[2]
# station_name = station_info.get_text()
# 如果有换乘
station_exchange_info = station_info.find('div',class_="tips-info-box")
station_exchange = station_exchange_info.find('a').get_text()
station_name = station_info.find("a",class_="link").get_text()
# station_exchange_info
l1.find("strong").get_text()

'深圳地铁1号线(罗宝线)线路图'

In [371]:
class station:
    def __init__(self,name,exchange,index):
        self.name = name
        self.exchange = exchange
        self.id = index
    def tolist(self):
        return [self.name, self.exchange, self.id]
    
dic = defaultdict(list)
for line in all_line:
    # 是否有换乘
    stations_info = line.find_all('div',class_="station")
    line_name = line.find("strong").get_text()[2:-3]
    index = 0
    for station_info in stations_info:
        if station_info.find('div',class_="tips-info-box"):
            station_exchange_info = station_info.find('div',class_="tips-info-box")
            station_exchange = station_exchange_info.find('a').get_text()
            station_name = station_info.find("a",class_="link").get_text()
            dic[line_name].append(station(station_name,station_exchange,index))
            
        else:
            station_name = station_info.get_text()
            station_exchange = "None"
            dic[line_name].append(station(station_name,station_exchange,index))
        index += 1


In [372]:
class line:
    def __init__(self,id,station_list):
        self.id = id
        self.stations = station_list
        self.names = self.station_names(station_list)
        self.exchange = self.exchange_func(station_list)
        
    def inline(self, station_name):
        return station_name in self.names
    
    def station_names(self,station_list):
        return[station.name for station in station_list]
    
    def reachable(self, name1,name2):
        return abs(self.names.index(name1)-self.names.index(name2))==1 or abs(self.names.index(name1)-self.names.index(name2))==len(self.names)-1
    
    def exchange_func(self,station_list):
        return [station.name for station in station_list if station.exchange != "None"]
    
    def get_path(self,name1,name2):
        if self.inline(name1) and self.inline(name2):
            index1 = self.names.index(name1)
            index2 = self.names.index(name2)
            path1 = [name1]
            path2 = [name1]
            for index in range(index1+1, len(self.names)):
                path1.append(self.names[index])
                if index == index2:
                    break
            for index in range(index1+1, len(self.names)):
                path2.append(self.names[len(self.names)-index])
                if len(self.names)-index == index2:
                    break
            return path1, path2
               
                    
                    
                
                    
        
    
lines = []
for line_item in dic.items():
    l = []
    for stat in line_item[1]:
#         print(stat.exchange)
        l.append(stat)
    line_obj = line(line_item[0],l)
    lines.append(line_obj)
    print(f"{line_item[0]} done")

地铁1号线(罗宝线) done
地铁2号线(蛇口线) done
地铁3号线(龙岗线) done
地铁4号线(龙华线) done
地铁5号线(环中线) done
地铁6号线(光明线) done
地铁7号线(西丽线) done
地铁9号线(梅林线) done
地铁10号线(坂田线) done
地铁11号线(机场线) done
龙华有轨电车示范线 done
龙华有轨电车示范线（支线） done


In [373]:
line1 = lines[0]
print(line1.names)
print(line1.exchange)
print(line1.inline("罗湖"))
print(line1.reachable("罗湖","国贸"))
print(line1.reachable("罗湖","机场东"))
print(line1.reachable("罗湖","大剧院"))
print(line1.get_path("罗湖","大剧院"))

['罗湖', '国贸', '老街', '大剧院', '科学馆', '华强路', '岗厦', '会展中心', '购物公园', '香蜜湖', '车公庙', '竹子林', '侨城东', '华侨城', '世界之窗', '白石洲', '高新园', '深大', '桃园', '大新', '鲤鱼门', '前海湾', '新安', '宝安中心', '宝体', '坪洲', '西乡', '固戍', '后瑞', '机场东']
['老街', '大剧院', '会展中心', '购物公园', '车公庙', '世界之窗', '前海湾', '宝安中心']
True
True
True
False
(['罗湖', '国贸', '老街', '大剧院'], ['罗湖', '机场东', '后瑞', '固戍', '西乡', '坪洲', '宝体', '宝安中心', '新安', '前海湾', '鲤鱼门', '大新', '桃园', '深大', '高新园', '白石洲', '世界之窗', '华侨城', '侨城东', '竹子林', '车公庙', '香蜜湖', '购物公园', '会展中心', '岗厦', '华强路', '科学馆', '大剧院'])


## 构建地铁网络图

In [375]:
class network:
    def __init__(self, line_list):
        self.lines = line_list
        self.exchange_point = self.get_exchange(line_list)
        self.vertexs = self.get_vertex(line_list)
        
    
    def get_line(self, station_name):
        lines = []
        for line in self.lines:
            if station_name in line.names:
                lines.append(line.id)
        return lines
    
    def get_exchange(self, line_list):
        dic = {}
        for line in line_list:
            dic[line.id] = line.exchange
        return dic
    
    def get_vertex(self,line_list):
        vert = defaultdict(list)
        for line in line_list:
            stations_name = line.names
            for i, st_name in enumerate(stations_name):
                
                if i+1 < len(stations_name):
#                     print(st_name,[stations_name[i+1],stations_name[i-1]])
                    vert[st_name] += [stations_name[i+1],stations_name[i-1]]
                else:
#                     print(st_name,[stations_name[0],stations_name[i-1]])
                    vert[st_name] += [stations_name[0],stations_name[i-1]]
        return vert
subway_net= network(lines)
subway_net.vertexs
# subway_net.get_line("老街")

defaultdict(list,
            {'罗湖': ['国贸', '机场东'],
             '国贸': ['老街', '罗湖'],
             '老街': ['大剧院', '国贸', '晒布', '红岭'],
             '大剧院': ['科学馆', '老街', '湖贝', '燕南'],
             '科学馆': ['华强路', '大剧院', '通新岭', '松岗'],
             '华强路': ['岗厦', '科学馆'],
             '岗厦': ['会展中心', '华强路', '岗厦北', '福民'],
             '会展中心': ['购物公园', '岗厦', '福民', '市民中心'],
             '购物公园': ['香蜜湖', '会展中心', '福田', '石厦'],
             '香蜜湖': ['车公庙', '购物公园'],
             '车公庙': ['竹子林', '香蜜湖', '上沙', '农林', '香梅', '下沙', '红树湾南', '福田'],
             '竹子林': ['侨城东', '车公庙'],
             '侨城东': ['华侨城', '竹子林'],
             '华侨城': ['世界之窗', '侨城东'],
             '世界之窗': ['白石洲', '华侨城', '侨城北', '红树湾'],
             '白石洲': ['高新园', '世界之窗'],
             '高新园': ['深大', '白石洲'],
             '深大': ['桃园', '高新园'],
             '桃园': ['大新', '深大'],
             '大新': ['鲤鱼门', '桃园'],
             '鲤鱼门': ['前海湾', '大新'],
             '前海湾': ['新安', '鲤鱼门', '桂湾', '临海', '宝安', '南山'],
             '新安': ['宝安中心', '前海湾'],
             '

## 改进版BFS

In [376]:
def search_2(graph,start,destination,search_strategy):
    pathes = [[start]]
    visited = set()# ！
    while pathes:
        path = pathes.pop(0)
        froniter = path[-1]
        if froniter in visited : continue# ！
            
        if froniter == destination:# ！
            return path# ！
        
        successsors = graph.vertexs[froniter]
        
        for city in successsors:
            if city in path: continue  # check loop
            
            new_path = path+[city]
            
            pathes.append(new_path)  #bfs
            
        pathes = search_strategy(graph, pathes)
        visited.add(froniter) # ！
       # if pathes and (destination == pathes[0][-1]):
       #     return pathes[0]  

## 两种搜索策略： 站数最少的， 换乘最少的

In [377]:
def sort_by_distance(graph,pathes):
    def get_distance_of_path(path):
        distance = 0
        for i,_ in enumerate(path[:-1]):
            distance += 1
        return distance
    return sorted(pathes,key=get_distance_of_path)

In [378]:
def sort_by_change(graph,pathes):
    def get_change_of_path(path):
        change = 0 
        ori_station = graph.get_line(path[0])[0]
        for st in path:
            if ori_station in graph.get_line(st):
                continue
            change += 1
            ori_station = graph.get_line(st)[0]
        return change
    return sorted(pathes,key=get_change_of_path)

In [383]:
# 最少换乘
best_path = search_2(subway_net,"人民南","洪湖",search_strategy=sort_by_change)
print(f"{'->'.join(best_path)}")

人民南->鹿丹村->红岭南->红岭->园岭->红岭北->笋岗->洪湖


In [384]:
# 最少站数
best_path = search_2(subway_net,"人民南","洪湖",search_strategy=sort_by_distance)
print(f"{'->'.join(best_path)}")

人民南->鹿丹村->红岭南->红岭->园岭->红岭北->笋岗->洪湖
